#Setting Up Chameleon Environment for Serving
- site being used here is
- Model Serving in

In [ ]:
from chi import server, context, lease
import os

context.version = "1.0"
context.choose_project()
context.choose_site(default="CHI@TACC")

In [ ]:
l = lease.get_lease(f"serve_model_netID")
l.show()

In [ ]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-serve-model-{username}",
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

In [ ]:
s.associate_floating_ip()

In [ ]:
s.refresh()
s.check_connectivity()

In [ ]:
s.refresh()
s.show(type="widget")

#Git Clone LLM Summarization



In [ ]:
s.execute("https://github.com/shettynitis/LLM_LegalDocSummarization.git")

#Setting Up Docker

In [ ]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

Set up the NVIDIA container toolkit

In [ ]:
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\") then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json")
s.execute("sudo systemctl restart docker")

#Open an SSH session

Finally, open an SSH sesson on your server. From your local terminal, run

``ssh -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D``

where,

in place of ~/.ssh/id_rsa_chameleon, substitute the path to our project key that you had uploaded

in place of A.B.C.D, use the floating IP address you just associated to your instance.

#Prepare data

- Create a volume
````
# runs on node
docker volume create llm-data
````

- Populate it with data
````
# runs on node
docker compose -f serve-model-chi/docker/docker-compose-data.yaml up -d
````

- To check if the container is still running
````
# runs on node-serve-model
docker ps
````
- Start a shell in a temporary container with this volume attached
````
# run on node‑mltrain (or whatever node you leased)
docker run --rm -it -v llm-data:/mnt alpine ls -l /mnt/merged_dataset/
````

#Launch a Jupyter container

````
# run on node server
docker build -t jupyter-onnx -f LLM_LegalDocSummarization/docker/Dockerfile.jupyter-onnx-gpu .

````


````
# run on node server
docker run  -d --rm  -p 8888:8888 \
    --gpus all \
    --shm-size 16G \
    -v ~/LLM_LegalDocSummarization:/home/jovyan/work \
    -v llm-data:/mnt/ \
    -e LLM_DATA_DIR=/mnt/merged_dataset \
    --name jupyter \
    jupyter-onnx

````

````
# run on node server
docker logs jupyter
````

and look for a line like
````
http://127.0.0.1:8888/lab?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
````